In [ ]:
# Pendências
# 1 - Legenda do Gráfico Geral descrições das contas
# 2 - Inserir no caderno a Função para baixar automaticamente os arquivos da API do SICONFI
# 3 - Definir os tópicos da apresentação
# 4 - Criar a apresentação
# 5 - Criar caderno com os comentários com markdown
# 6 - Contas que não aparecem em balanco_final_exercicio
# - OBRIGAÇÕES TRABALHISTAS, PREVIDENCIÁRIAS E ASSISTENCIAIS A PAGAR A CURTO PRAZO * não aparece em balanco_final_exercicio
# - RESULTADOS ACUMULADOS -  não aparece em balanco_final_exercicio

In [ ]:
import os, pandas as pd, re
from datetime import date
import numpy as np
from sklearn import preprocessing
import numpy as np


def build_path(subfolder = 'data'):
    folderpath = os.path.join(os.getcwd(), 
                              'projeto', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath


def get_dados_pcasp(
        filename='CPU_PCASP_2022.xlsx'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_excel(filepath, sheet_name= 'Federação 2022')


pcasp = get_dados_pcasp()

pcasp = pcasp.drop(columns=['FUNÇÃO','STATUS','NATUREZA DO SALDO','CONTROLE','ÍTEM','SUBÍTEM'])

pcasp = pcasp.loc[pcasp['CLASSE'].isin([1, 2])]

pcasp.loc[pcasp['TÍTULO.1'] == 'OBRIGAÇÕES TRABALHISTAS, PREVIDENCIÁRIAS E ASSISTENCIAIS A PAGAR A LONGO PRAZO','TÍTULO.1'] = 'OBRIGAÇÕES TRAB., PREV. E ASSIST. A PAGAR A LONGO PRAZO'

pcasp.loc[pcasp['TÍTULO.1'] == 'OBRIGAÇÕES TRABALHISTAS, PREVIDENCIÁRIAS E ASSISTENCIAIS A PAGAR A CURTO PRAZO','TÍTULO.1'] = 'OBRIGAÇÕES TRAB., PREV. E ASSIST. A PAGAR A CURTO PRAZO'

pcasp['NÍVEL3'] = pcasp['CLASSE'].astype(str) + pcasp['GRUPO'].astype(str) + pcasp['SUBGRUPO'].astype(str)

pcasp['NÍVEL2'] = pcasp['CLASSE'].astype(str) + pcasp['GRUPO'].astype(str)

pcasp['NÍVEL1'] = pcasp['CLASSE'].astype(str)

pcasp_nivel_3 = pcasp.loc[(pcasp['SUBGRUPO']!= 0) & (pcasp['TÍTULO']== 0)]

pcasp_nivel_2 = pcasp.loc[(pcasp['GRUPO']!= 0) & (pcasp['SUBGRUPO']== 0)]

pcasp_nivel_1 = pcasp.loc[(pcasp['GRUPO']== 0)]

pcasp_nivel_1_2_3 = pd.concat([pcasp_nivel_1,pcasp_nivel_2,pcasp_nivel_3],axis=0)

build_path()


def get_dados_instancia(filename='instancia_MSC_API.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath, sep=';')

instancia = get_dados_instancia()

instancia = instancia.drop(columns=['tipo_matriz','cod_ibge','poder_orgao','financeiro_permanente','ano_fonte_recursos','fonte_recursos','mes_referencia','divida_consolidada','data_referencia','entrada_msc','tipo_valor','complemento_fonte'])

instancia = instancia.rename(columns={'classe_conta':'NÍVEL1', 'conta_contabil':'CONTA', 'exercicio': 'EXERCÍCIO', 'valor':'VALOR', 'natureza_conta':'NATUREZA_VALOR'})

instancia['CONTA'] = instancia['CONTA'].astype(str)

#  Multiplica por 1 - Desnecessária
instancia.loc[instancia['NÍVEL1'].isin([1]) & 
              (instancia['NATUREZA_VALOR']=='D'), 'VALOR'] = instancia['VALOR']*1

#  Multiplica por -1
instancia.loc[instancia['NÍVEL1'].isin([1]) & 
              (instancia['NATUREZA_VALOR']=='C'), 'VALOR'] = instancia['VALOR']*(-1)

#  Multiplica por 1 - Desnecessária
instancia.loc[instancia['NÍVEL1'].isin([2]) & 
              (instancia['NATUREZA_VALOR']=='C'), 'VALOR'] = instancia['VALOR']*1

#  Multiplica por -1
instancia.loc[instancia['NÍVEL1'].isin([2]) & 
              (instancia['NATUREZA_VALOR']=='D'), 'VALOR'] = instancia['VALOR']*(-1)

instancia['NÍVEL3'] = instancia['CONTA'].str.slice(0, 3)
instancia['NÍVEL2'] = instancia['CONTA'].str.slice(0,2)


coluna = [col for col in instancia if col != 'NÍVEL1'] + ['NÍVEL1']

instancia=instancia[coluna]

instancia_grouped = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL3'], instancia['NATUREZA_VALOR']])

instancia_soma_nivel_3 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL3']]).sum()

instancia_soma_nivel_2 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL2']]).sum()

instancia_soma_nivel_1 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL1']]).sum()

instancia_soma_nivel_3=instancia_soma_nivel_3.reset_index(level=['NÍVEL3','EXERCÍCIO'])

instancia_soma_nivel_2 = instancia_soma_nivel_2.reset_index(level=['NÍVEL2','EXERCÍCIO'])

instancia_soma_nivel_1 = instancia_soma_nivel_1.reset_index(level=['NÍVEL1','EXERCÍCIO'])

balanco_nivel_3 = pd.merge(pcasp_nivel_3, instancia_soma_nivel_3, how='inner')

balanco_nivel_3 = balanco_nivel_3.sort_values(by=['EXERCÍCIO','CONTA'])

balanco_nivel_3 = balanco_nivel_3.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

balanco_nivel_2 = pd.merge(pcasp_nivel_2, instancia_soma_nivel_2, how='inner')

balanco_nivel_2 = balanco_nivel_2.sort_values(by=['EXERCÍCIO','CONTA'])

balanco_nivel_2 = balanco_nivel_2.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

instancia_soma_nivel_1['NÍVEL1'] = instancia_soma_nivel_1['NÍVEL1'].astype(str)

balanco_nivel_1 = pd.merge(pcasp_nivel_1, instancia_soma_nivel_1, how='inner')

balanco_nivel_1 = balanco_nivel_1.sort_values(by=['EXERCÍCIO','CONTA'])

balanco_nivel_1 = balanco_nivel_1.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

balanco_123 = pd.concat([balanco_nivel_1,balanco_nivel_2,balanco_nivel_3],axis=0)

balanco_123 = balanco_123.sort_values(by=['EXERCÍCIO','CONTA'])


In [ ]:
balanco_123 = balanco_123.drop(columns=['NÍVEL3','NÍVEL2','NÍVEL1'])
lista_exercicios = list(set(balanco_123['EXERCÍCIO']))
balanco = []
for ano in lista_exercicios:
    balanco_ano = balanco_123.loc[balanco_123['EXERCÍCIO'] == ano]
    balanco_ano = balanco_ano.rename(columns={'VALOR': f'{ano}'})
    balanco_ano = balanco_ano.drop(columns=['EXERCÍCIO'])
    balanco.append(balanco_ano)
balanco_final = pd.merge(balanco[0], balanco[1], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = pd.merge(balanco_final, balanco[2], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = pd.merge(balanco_final, balanco[3], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = balanco_final.sort_values(by='CONTA')

In [ ]:
balanco_final_exercicio = balanco_final.T

In [ ]:
lista = []
for i in range(len(balanco_final)):
    lista.append( list(balanco_final_exercicio.loc['TÍTULO.1'])[i])

In [ ]:
balanco_final_exercicio = balanco_final_exercicio.rename(columns=balanco_final_exercicio.iloc[1])
balanco_final_exercicio = balanco_final_exercicio[2:]
balanco_final_exercicio.index.names = ['EXERCÍCIO']
balanco_final_exercicio = balanco_final_exercicio.reset_index(level=['EXERCÍCIO'])

In [ ]:
balanco_final['CONTA'] = balanco_final['CONTA'].astype(str)
balanco_final['CLASSE'] = balanco_final['CONTA'].str.slice(0,1)

balanco_final = balanco_final.rename(columns={2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022'})

In [ ]:
balanco_normalizado = balanco_final.copy()

In [ ]:
listaAnos = ['2019', '2020', '2021', '2022' ]
for ano in listaAnos:
    coluna_ano = listaAnos[listaAnos.index(ano)]

    if balanco_final[coluna_ano].min() < 0 :

        listanorm = preprocessing.normalize([balanco_normalizado[coluna_ano].values.tolist()]) 
        coluna_ano = coluna_ano+'ABS'
        listanormal = listanorm-(listanorm.min()*2)
        balanco_normalizado[coluna_ano] = pd.DataFrame(listanormal).T
